In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

#branch_names = '''momentum,energy,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
#TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
#DiscThetaC,DiscQuality,
#EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
#EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
#DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = '''momentum,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,
TofStopTime,TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)
###################################################################################

electrons = root2array('/home/wesmail/Downloads/150k/electron_150k_tree.root', 't1', branch_names)
electrons = rec2array(electrons)

pions = root2array('/home/wesmail/Downloads/150k/pion_150k_tree.root', 't1', branch_names)
pions = rec2array(pions)

muons = root2array('/home/wesmail/Downloads/150k/muon_150k_tree.root', 't1', branch_names)
muons = rec2array(muons)

kaons = root2array('/home/wesmail/Downloads/150k/kaon_150k_tree.root', 't1', branch_names)
kaons = rec2array(kaons)

anti_p = root2array('/home/wesmail/Downloads/150k/proton_150k_tree.root', 't1', branch_names)
anti_p = rec2array(anti_p)

###################################################################################
X = np.concatenate((electrons, pions, muons, kaons, anti_p))
y = np.concatenate(( np.zeros(electrons.shape[0]),np.ones(pions.shape[0]), (2*np.ones(muons.shape[0])), (3*np.ones(kaons.shape[0])), (4*np.ones(anti_p.shape[0])) ))
df = pd.DataFrame(np.hstack((X, y.reshape(y.shape[0], -1))),columns=branch_names+['temp'])

# added features
#df['E1/E9'] = df.loc[:,'EmcE1']/df.loc[:,'EmcE9']
#df['E9/E25'] = df.loc[:,'EmcE9']/df.loc[:,'EmcE25']
#df['E1/E25'] = df.loc[:,'EmcE1']/df.loc[:,'EmcE25']
df['E/p'] = df.loc[:,'EmcCalEnergy']/df.loc[:,'momentum']
#df['Pt'] = np.sqrt( df.loc[:,'momentumx']**2  + df.loc[:,'momentumy']**2 )
#df = df.drop(['momentumx'], axis=1)
#df = df.drop(['momentumy'], axis=1)
df['labels'] = df.loc[:,'temp']
df = df.drop(['temp'], axis=1)

#df = df.dropna()
#df.tail(5)
print (len(df.columns))

37



variable "df" not defined.

variable "len(df.col))" not defined.


In [2]:
%matplotlib tk
f, ax = plt.subplots(figsize=(8, 6))
plt.title('Feature Correlations (BoxGen)')
corr = df.iloc[:,0:-1].corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), xticklabels=corr.columns, yticklabels=corr.columns,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.30, random_state=0)
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

bdt = RandomForestClassifier(n_estimators=10, max_depth=5, max_features=35, random_state=0, n_jobs=4)
xgb = XGBClassifier(max_depth=12, learning_rate=0.15, n_jobs=4)
%time model = xgb.fit(X_train, y_train)
print('accuracy on training set: %.2f' % (model.score(X_train, y_train)*100) + '%')
print('accuracy on testing set : %.2f' % (model.score(X_test, y_test)*100) + '%')

import matplotlib.pyplot as plt
%matplotlib tk

CPU times: user 22min 42s, sys: 486 ms, total: 22min 43s
Wall time: 5min 41s


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy on training set: 95.94%
accuracy on testing set : 88.10%


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
from sklearn.externals import joblib

import pickle
joblib.dump(model, 'saved_BDT_Box_.pkl')

['saved_BDT_Box_.pkl']

In [9]:
from sklearn.model_selection import validation_curve

depth = np.arange(10,80,10)
#print(AdaBoostClassifier().get_params().keys())
train_score, val_score = validation_curve(RandomForestClassifier(n_estimators=10, max_features=35, random_state=0, n_jobs=4), X_train, y_train, param_name='n_estimators', param_range=depth)

In [7]:
fig, axs = plt.subplots()
plt.plot(depth, np.median(train_score, 1), color='blue', linewidth= 2, label='Training Score')
plt.plot(depth, np.median(val_score, 1), color='red', linewidth= 2, label='Testing Score')
axs.annotate('', xy=(15, 0.87), xytext=(15, 0.6),
arrowprops=dict(facecolor='black', shrink=0.05))
plt.legend(loc='best', fontsize=20)
plt.ylim(0, 1)
plt.xlabel('Depth', fontsize=20)
plt.ylabel('Score', fontsize=20)
plt.grid(True)
plt.gray()

NameError: name 'depth' is not defined


variable "plt" not defined.

variable "plt.ti" not defined.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

branch_names = '''momentumx, momentumy,momentumz,momentum,energy,position,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)
###################################################################################

electronsp = root2array('/home/wesmail/Downloads/150k/plus/Pelectron_theta_150k_tree.root', 't1', branch_names)
electronsp = rec2array(electronsp)

pionsp = root2array('/home/wesmail/Downloads/150k/plus/Ppion_theta_150k_tree.root', 't1', branch_names)
pionsp = rec2array(pionsp)

muonsp = root2array('/home/wesmail/Downloads/150k/plus/Pmuon_theta_150k_tree.root', 't1', branch_names)
muonsp = rec2array(muonsp)

kaonsp = root2array('/home/wesmail/Downloads/150k/plus/Pkaon_theta_150k_tree.root', 't1', branch_names)
kaonsp = rec2array(kaonsp)

anti_pp = root2array('/home/wesmail/Downloads/150k/plus/Pproton_theta_150k_tree.root', 't1', branch_names)
anti_pp = rec2array(anti_pp)

###################################################################################
Xp = np.concatenate((electronsp, pionsp, muonsp, kaonsp, anti_pp))
yp = np.concatenate(( np.zeros(electronsp.shape[0]),np.ones(pionsp.shape[0]), (2*np.ones(muonsp.shape[0])), (3*np.ones(kaonsp.shape[0])), (4*np.ones(anti_pp.shape[0])) ))
dfp = pd.DataFrame(np.hstack((Xp, yp.reshape(yp.shape[0], -1))),columns=branch_names+['temp'])

# added features
dfp['E1/E9'] = dfp.loc[:,'EmcE1']/dfp.loc[:,'EmcE9']
dfp['E9/E25'] = dfp.loc[:,'EmcE9']/dfp.loc[:,'EmcE25']
dfp['E1/E25'] = dfp.loc[:,'EmcE1']/dfp.loc[:,'EmcE25']
dfp['E/p'] = dfp.loc[:,'EmcCalEnergy']/dfp.loc[:,'momentum']
dfp['labels'] = dfp.loc[:,'temp']
dfp = dfp.drop(['temp'], axis=1)

dfp = dfp.dropna()

In [4]:
print((model.predict(X_test)).shape)

(150193,)



tab completion not implemented for this context

variable "model" not defined.

variable "print(model.pred)" not defined.

variable "model" not defined.

variable "print(model.predict(X_tes))" not defined.


In [7]:
model.classes_
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

figcm, ax = plt.subplots()
y_predicted = model.predict(X_test)
cm = confusion_matrix(y_test, y_predicted)
cm = np.round((cm.astype('float') / cm.sum(axis=1)), decimals=2)
sns.set(font_scale=2.0)
sns.heatmap(cm, square=True, annot=True, annot_kws={"size": 22}, cmap='Blues')
#sns.heatmap(cm, square=True, annot=True, cbar=False, annot_kws={"size": 20})
classes=['$e^{\pm}$', '$\pi^{\pm}$', '$\mu^{\pm}$', '$k^{\pm}$', '$\overline{p}$'] # 0 1 2 3 4 output of 'model.classes_'
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=90)
ax.set_xticks(np.arange(len(classes))+0.5, minor=False)
plt.yticks(tick_marks, classes)
ax.set_yticks(np.arange(len(classes))+0.5, minor=False)
plt.xlabel('Predication', horizontalalignment = 'center',  fontsize=22)
plt.ylabel('True Species',  fontsize=22)
plt.title('Particles',  fontsize=22)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
features = list(df.columns)
features = features[0:-1]
print(features)
plt.plot(model.feature_importances_, 'o', color='red', markersize=15)
plt.xticks(range(len(features)), features, rotation=90 )
plt.ylim(0,0.4)
plt.ylabel('Importance')
plt.title('|$\overrightarrow{p}$| =  0.2 - 5 GeV/c', fontsize=22)

['momentum', 'MvdDEDX', 'MvdHits', 'SttMeanDEDX', 'SttHits', 'GemHits', 'TofStopTime', 'TofM2', 'TofTrackLength', 'TofQuality', 'TofBeta', 'DrcThetaC', 'DrcQuality', 'DiscThetaC', 'DiscQuality', 'EmcRawEnergy', 'EmcCalEnergy', 'EmcQuality', 'EmcNumberOfCrystals', 'EmcNumberOfBumps', 'EmcLat', 'MuoQuality', 'MuoIron', 'MuoMomentumIn', 'MuoNumberOfLayers', 'MuoModule', 'MuoHits', 'ChiSquared', 'E/p', 'Pt']


In [5]:
from sklearn.externals import joblib

model = joblib.load('saved_BDT_Box_.pkl')


from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.30, random_state=0)

In [6]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
y_score = model.predict_proba(X_test)
from sklearn.preprocessing import label_binarize
y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
n_classes = y_test.shape[1]
import matplotlib.pyplot as plt
%matplotlib tk

lw=2
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

from itertools import cycle

plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$e^{-}$',roc_auc[0]))
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\pi^{-}$',roc_auc[1]))
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\mu^{-}$',roc_auc[2]))
plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$k^{-}$',roc_auc[3]))
plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='ROC for {0}   (area = {1:0.2f})'.format('$\overline{p}$',roc_auc[4]))

plt.xlim([0.0, 0.5])
plt.ylim([0.5, 1.0])
plt.xlabel('False Positive Rate', fontsize=22)
plt.ylabel('True Positive Rate', fontsize=22)
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right', fontsize=22)    

In [7]:
from sklearn.metrics import precision_recall_curve, average_precision_score 

precisions = dict()
recalls = dict()
pr_auc = dict()
for i in range(n_classes):
    precisions[i], recalls[i], _ = precision_recall_curve(y_test[:,i], y_score[:,i])
    pr_auc[i] = average_precision_score(y_test[:,i], y_score[:,i])

plt.plot(recalls[0], precisions[0], color='aqua', lw=lw, label='{0} (area = {1:0.2f})'.format('$e^{-}$',pr_auc[0]))
plt.plot(recalls[1], precisions[1], color='darkorange', lw=lw, label='{0} (area = {1:0.2f})'.format('$\pi^{-}$',pr_auc[1]))
plt.plot(recalls[2], precisions[2], color='cornflowerblue', lw=lw, label='{0} (area = {1:0.2f})'.format('$\mu^{-}$',pr_auc[2]))
plt.plot(recalls[3], precisions[3], color='black', lw=lw, label='{0} (area = {1:0.2f})'.format('$k^{-}$',pr_auc[3]))
plt.plot(recalls[4], precisions[4], color='green', lw=lw, label='{0} (area = {1:0.2f})'.format('$\overline{p}$',pr_auc[4]))

plt.xlim([0.3, 1.0])
plt.ylim([0.3, 1.0])
plt.xlabel('Recalls', fontsize=22)
plt.ylabel('Precisions', fontsize=22)
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left', fontsize=22)  

In [104]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

branch_names = """momentumx, momentumy,momentumz,momentum,energy,position,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared""".split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)

electrons1 = root2array("/home/wesmail/Downloads/MS/electronMS1_tree.root", "t1", branch_names)
electrons1 = rec2array(electrons1)
electrons2 = root2array("/home/wesmail/Downloads/MS/electronMS2_tree.root", "t1", branch_names)
electrons2 = rec2array(electrons2)
electrons3 = root2array("/home/wesmail/Downloads/MS/electronMS3_tree.root", "t1", branch_names)
electrons3 = rec2array(electrons3)

pions1 = root2array("/home/wesmail/Downloads/MS/pionMS1_tree.root", "t1", branch_names)
pions1 = rec2array(pions1)
pions2 = root2array("/home/wesmail/Downloads/MS/pionMS2_tree.root", "t1", branch_names)
pions2 = rec2array(pions2)
pions3 = root2array("/home/wesmail/Downloads/MS/pionMS3_tree.root", "t1", branch_names)
pions3 = rec2array(pions3)


muons1 = root2array("/home/wesmail/Downloads/MS/muonMS1_tree.root", "t1", branch_names)
muons1 = rec2array(muons1)
muons2 = root2array("/home/wesmail/Downloads/MS/muonMS2_tree.root", "t1", branch_names)
muons2 = rec2array(muons2)
muons3 = root2array("/home/wesmail/Downloads/MS/muonMS3_tree.root", "t1", branch_names)
muons3 = rec2array(muons3)

kaons1 = root2array("/home/wesmail/Downloads/MS/kaonMS1_tree.root", "t1", branch_names)
kaons1 = rec2array(kaons1)
kaons2 = root2array("/home/wesmail/Downloads/MS/kaonMS2_tree.root", "t1", branch_names)
kaons2 = rec2array(kaons2)
kaons3 = root2array("/home/wesmail/Downloads/MS/kaonMS3_tree.root", "t1", branch_names)
kaons3 = rec2array(kaons3)

anti_p1 = root2array("/home/wesmail/Downloads/MS/protonMS1_tree.root", "t1", branch_names)
anti_p1 = rec2array(anti_p1)
anti_p2 = root2array("/home/wesmail/Downloads/MS/protonMS2_tree.root", "t1", branch_names)
anti_p2 = rec2array(anti_p2)
anti_p3 = root2array("/home/wesmail/Downloads/MS/protonMS3_tree.root", "t1", branch_names)
anti_p3 = rec2array(anti_p3)

X1 = np.concatenate((electrons1, pions1, muons1, kaons1, anti_p1))
y1 = np.concatenate(( np.zeros(electrons1.shape[0]),np.ones(pions1.shape[0]), (2*np.ones(muons1.shape[0])), (3*np.ones(kaons1.shape[0])), (4*np.ones(anti_p1.shape[0])) ))

X2 = np.concatenate((electrons2, pions2, muons2, kaons2, anti_p2))
y2 = np.concatenate(( np.zeros(electrons2.shape[0]),np.ones(pions2.shape[0]), (2*np.ones(muons2.shape[0])), (3*np.ones(kaons2.shape[0])), (4*np.ones(anti_p2.shape[0])) ))

X3 = np.concatenate((electrons3, pions3, muons3, kaons3, anti_p3))
y3 = np.concatenate(( np.zeros(electrons3.shape[0]),np.ones(pions3.shape[0]), (2*np.ones(muons3.shape[0])), (3*np.ones(kaons3.shape[0])), (4*np.ones(anti_p3.shape[0])) ))

df1 = pd.DataFrame(np.hstack((X1, y1.reshape(y1.shape[0], -1))),columns=branch_names+['temp'])
df2 = pd.DataFrame(np.hstack((X2, y2.reshape(y2.shape[0], -1))),columns=branch_names+['temp'])
df3 = pd.DataFrame(np.hstack((X3, y3.reshape(y3.shape[0], -1))),columns=branch_names+['temp'])

# added features
df1['momentum'] = np.sqrt(df1.loc[:,'momentum'])
df1['E1/E9'] = df1.loc[:,'EmcE1']/df1.loc[:,'EmcE9']
df1['E9/E25'] = df1.loc[:,'EmcE9']/df1.loc[:,'EmcE25']
df1['E1/E25'] = df1.loc[:,'EmcE1']/df1.loc[:,'EmcE25']
df1['E/p'] = df1.loc[:,'EmcCalEnergy']/df1.loc[:,'momentum']
df1['labels'] = df1.loc[:,'temp']
df1 = df1.drop(['temp'], axis=1)

df1 = df1.dropna()


# added features
df2['momentum'] = np.sqrt(df2.loc[:,'momentum']) 
df2['E1/E9'] = df2.loc[:,'EmcE1']/df2.loc[:,'EmcE9']
df2['E9/E25'] = df2.loc[:,'EmcE9']/df2.loc[:,'EmcE25']
df2['E1/E25'] = df2.loc[:,'EmcE1']/df2.loc[:,'EmcE25']
df2['E/p'] = df2.loc[:,'EmcCalEnergy']/df2.loc[:,'momentum']
df2['labels'] = df2.loc[:,'temp']
df2 = df2.drop(['temp'], axis=1)

df2 = df2.dropna()

# added features
df3['momentum'] = np.sqrt(df3.loc[:,'momentum']) 
df3['E1/E9'] = df3.loc[:,'EmcE1']/df3.loc[:,'EmcE9']
df3['E9/E25'] = df3.loc[:,'EmcE9']/df3.loc[:,'EmcE25']
df3['E1/E25'] = df3.loc[:,'EmcE1']/df3.loc[:,'EmcE25']
df3['E/p'] = df3.loc[:,'EmcCalEnergy']/df3.loc[:,'momentum']
df3['labels'] = df3.loc[:,'temp']
df3 = df3.drop(['temp'], axis=1)

df3 = df3.dropna()
from sklearn.utils import shuffle
df1 = shuffle(df1)
df2 = shuffle(df2)
df3 = shuffle(df3)

In [9]:
model.classes_
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

figcm, ax = plt.subplots()
y_predicted = model.predict(df3.iloc[:,0:-1])
cm = confusion_matrix(df3.iloc[:,-1], y_predicted)
cm = np.round((cm.astype('float') / cm.sum(axis=1)), decimals=2)
sns.set(font_scale=2.0)
sns.heatmap(cm, square=True, annot=True, annot_kws={"size": 22}, cmap='Blues')
#sns.heatmap(cm, square=True, annot=True, cbar=False, annot_kws={"size": 20})
classes=['$e^{-}$', '$\pi^{-}$', '$\mu^{-}$', '$k^{-}$', '$\overline{p}$'] # 0 1 2 3 4 output of 'model.classes_'
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=90)
ax.set_xticks(np.arange(len(classes))+0.5, minor=False)
plt.yticks(tick_marks, classes)
ax.set_yticks(np.arange(len(classes))+0.5, minor=False)
plt.xlabel('Predication', horizontalalignment = 'center',  fontsize=22)
plt.ylabel('True Species',  fontsize=22)
plt.title('Particles',  fontsize=22)
plt.title('|$\overrightarrow{p}$| =  2 - 5 GeV/c', fontsize=22)

NameError: name 'df3' is not defined

In [92]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
e_df = pd.DataFrame(electrons3, columns=branch_names)
e_df['E1/E9'] = e_df.loc[:,'EmcE1']/e_df.loc[:,'EmcE9']
e_df['E9/E25'] = e_df.loc[:,'EmcE9']/e_df.loc[:,'EmcE25']
e_df['E1/E25'] = e_df.loc[:,'EmcE1']/e_df.loc[:,'EmcE25']
e_df['E/p'] = e_df.loc[:,'EmcCalEnergy']/e_df.loc[:,'momentum']
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
pi_df = pd.DataFrame(pions3, columns=branch_names)
pi_df['E1/E9'] = pi_df.loc[:,'EmcE1']/pi_df.loc[:,'EmcE9']
pi_df['E9/E25'] = pi_df.loc[:,'EmcE9']/pi_df.loc[:,'EmcE25']
pi_df['E1/E25'] = pi_df.loc[:,'EmcE1']/pi_df.loc[:,'EmcE25']
pi_df['E/p'] = pi_df.loc[:,'EmcCalEnergy']/pi_df.loc[:,'momentum']
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
mu_df = pd.DataFrame(muons3, columns=branch_names)
mu_df['E1/E9'] = mu_df.loc[:,'EmcE1']/mu_df.loc[:,'EmcE9']
mu_df['E9/E25'] = mu_df.loc[:,'EmcE9']/mu_df.loc[:,'EmcE25']
mu_df['E1/E25'] = mu_df.loc[:,'EmcE1']/mu_df.loc[:,'EmcE25']
mu_df['E/p'] = mu_df.loc[:,'EmcCalEnergy']/mu_df.loc[:,'momentum']
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
k_df = pd.DataFrame(kaons3, columns=branch_names)
k_df['E1/E9'] = k_df.loc[:,'EmcE1']/k_df.loc[:,'EmcE9']
k_df['E9/E25'] = k_df.loc[:,'EmcE9']/k_df.loc[:,'EmcE25']
k_df['E1/E25'] = k_df.loc[:,'EmcE1']/k_df.loc[:,'EmcE25']
k_df['E/p'] = k_df.loc[:,'EmcCalEnergy']/k_df.loc[:,'momentum']
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ap_df = pd.DataFrame(anti_p3, columns=branch_names)
ap_df['E1/E9'] = ap_df.loc[:,'EmcE1']/ap_df.loc[:,'EmcE9']
ap_df['E9/E25'] = ap_df.loc[:,'EmcE9']/ap_df.loc[:,'EmcE25']
ap_df['E1/E25'] = ap_df.loc[:,'EmcE1']/ap_df.loc[:,'EmcE25']
ap_df['E/p'] = ap_df.loc[:,'EmcCalEnergy']/ap_df.loc[:,'momentum']
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
e_df = e_df.dropna()
pi_df = pi_df.dropna()
mu_df = mu_df.dropna()
k_df = k_df.dropna()
ap_df = ap_df.dropna()

e_f = pd.DataFrame(model.predict_proba(e_df))
pi_f = pd.DataFrame(model.predict_proba(pi_df))
mu_f = pd.DataFrame(model.predict_proba(mu_df))
k_f = pd.DataFrame(model.predict_proba(k_df))
ap_f = pd.DataFrame(model.predict_proba(ap_df))


el = e_f.iloc[:,0].ravel()
pi = pi_f.iloc[:,1].ravel()
mu = mu_f.iloc[:,2].ravel()
ka = k_f.iloc[:,3].ravel()
ap = ap_f.iloc[:,4].ravel()


In [5]:
List = branch_names + ['E1/E9', 'E9/E25', 'E1/E25', 'E/p']
#print(pi)
#print(pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==4, List])))

In [6]:
from root_numpy import array2root


e_f = pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==0, List]))
pi_f = pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==1, List]))
mu_f = pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==2, List]))
k_f = pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==3, List]))
ap_f = pd.DataFrame(model.predict_proba(df.loc[df.loc[:,'labels']==4, List]))

el = e_f.iloc[:,0].ravel()
pi = pi_f.iloc[:,1].ravel()
mu = mu_f.iloc[:,2].ravel()
ka = k_f.iloc[:,3].ravel()
ap = ap_f.iloc[:,4].ravel()

momentumArr_el = np.array(df.loc[df.loc[:,'labels']==0, ['momentum']]).ravel()
momentumArr_pi = np.array(df.loc[df.loc[:,'labels']==1, ['momentum']]).ravel()
momentumArr_mu = np.array(df.loc[df.loc[:,'labels']==2, ['momentum']]).ravel()
momentumArr_ka = np.array(df.loc[df.loc[:,'labels']==3, ['momentum']]).ravel()
momentumArr_ap = np.array(df.loc[df.loc[:,'labels']==4, ['momentum']]).ravel()

el.dtype = [('ele_pro', np.float64)]
pi.dtype = [('pi_pro', np.float64)]
mu.dtype = [('mu_pro', np.float64)]
ka.dtype = [('ka_pro', np.float64)]
ap.dtype = [('ap_pro', np.float64)]
momentumArr_el.dtype = [('electron_momentum', np.float64)]
momentumArr_pi.dtype = [('pion_momentum', np.float64)]
momentumArr_mu.dtype = [('muon_momentum', np.float64)]
momentumArr_ka.dtype = [('kaon_momentum', np.float64)]
momentumArr_ap.dtype = [('proton_momentum', np.float64)]


array2root(momentumArr_el, 'test-prediction.root', 'electrons', mode='recreate')
array2root(momentumArr_pi, 'test-prediction.root', 'pions')
array2root(momentumArr_mu, 'test-prediction.root', 'muons')
array2root(momentumArr_ka, 'test-prediction.root', 'kaons')
array2root(momentumArr_ap, 'test-prediction.root', 'protons')
array2root(el, 'test-prediction.root', 'electrons', mode='update')
array2root(pi, 'test-prediction.root', 'pions')
array2root(mu, 'test-prediction.root', 'muons')
array2root(ka, 'test-prediction.root', 'kaons')
array2root(ap, 'test-prediction.root', 'protons')

In [94]:
momentumArr_el = e_df.loc[:,'momentum'].ravel()
momentumArr_pi = pi_df.loc[:,'momentum'].ravel()
momentumArr_mu = mu_df.loc[:,'momentum'].ravel()
momentumArr_ka = k_df.loc[:,'momentum'].ravel()
momentumArr_ap = ap_df.loc[:,'momentum'].ravel()

el.dtype = [('ele_pro', np.float64)]
pi.dtype = [('pi_pro', np.float64)]
mu.dtype = [('mu_pro', np.float64)]
ka.dtype = [('ka_pro', np.float64)]
ap.dtype = [('ap_pro', np.float64)]
momentumArr_el.dtype = [('electron_momentum', np.float64)]
momentumArr_pi.dtype = [('pion_momentum', np.float64)]
momentumArr_mu.dtype = [('muon_momentum', np.float64)]
momentumArr_ka.dtype = [('kaon_momentum', np.float64)]
momentumArr_ap.dtype = [('proton_momentum', np.float64)]


array2root(momentumArr_el, 'test-prediction.root', 'electrons', mode='recreate')
array2root(momentumArr_pi, 'test-prediction.root', 'pions')
array2root(momentumArr_mu, 'test-prediction.root', 'muons')
array2root(momentumArr_ka, 'test-prediction.root', 'kaons')
array2root(momentumArr_ap, 'test-prediction.root', 'protons')
array2root(el, 'test-prediction.root', 'electrons', mode='update')
array2root(pi, 'test-prediction.root', 'pions')
array2root(mu, 'test-prediction.root', 'muons')
array2root(ka, 'test-prediction.root', 'kaons')
array2root(ap, 'test-prediction.root', 'protons')

In [95]:
import ROOT
infile = ROOT.TFile.Open('test-prediction.root', 'read')
infile.cd()
electrons_tree = infile.Get('electrons')
pions_tree = infile.Get('pions')
muons_tree = infile.Get('muons')
kaons_tree = infile.Get('kaons')
protons_tree = infile.Get('protons')

#bins = np.array([0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3., 3.5, 4., 4.5, 5.0,]) # variable binning
#bins = np.array([ 0.2, 0.5, 0.6, 0.7, 0.8, 0.9, 1.,]) # variable binning
#bins = np.array([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.8, 2.0,]) # variable binning
bins = np.array([2, 2.2, 2.5, 3.0, 3.5, 3.8, 4., 4.2, 4.5, 4.7, 5.0,]) # variable binning

# =================================================================================
histo_denE = ROOT.TH1F('denE', 'denE', len(bins)-1, bins)
histo_numE = ROOT.TH1F('numE', 'numE', len(bins)-1, bins)

electrons_tree.Draw('electron_momentum >> denE', 'electron_momentum>0')
electrons_tree.Draw('electron_momentum >> numE', 'electron_momentum>0 & ele_pro>0.5')

eff0 = ROOT.TEfficiency(histo_numE, histo_denE)
eff0.SetTitle('BDT efficiency e^{-}[ P(e^{-})>0.5 ]; momentum [GeV/c];#epsilon')
eff0.SetName('electron')
eff0.SetMarkerStyle(20)
eff0.SetLineColor(7)
eff0.SetMarkerSize(2)
eff0.SetMarkerColor(7)
eff0.Draw('APL')
# ==================================================================================

histo_denPi = ROOT.TH1F('denPi', 'denPi', len(bins)-1, bins)
histo_numPi = ROOT.TH1F('numPi', 'numPi', len(bins)-1, bins)

pions_tree.Draw('pion_momentum >> denPi', 'pion_momentum>0')
pions_tree.Draw('pion_momentum >> numPi', 'pion_momentum>0 & pi_pro>0.5')

eff1 = ROOT.TEfficiency(histo_numPi, histo_denPi)
eff1.SetTitle('BDT efficiency #pi^{-}[ P(#pi^{-})>0.5 ]; momentum [GeV/c];#epsilon')
eff1.SetName('pion')
eff1.SetMarkerStyle(21)
eff1.SetLineColor(800)
eff1.SetMarkerSize(2)
eff1.SetMarkerColor(800)
eff1.Draw('APL')
# ==================================================================================

histo_denM = ROOT.TH1F('denM', 'denM', len(bins)-1, bins)
histo_numM = ROOT.TH1F('numM', 'numM', len(bins)-1, bins)

muons_tree.Draw('muon_momentum >> denM', 'muon_momentum>0')
muons_tree.Draw('muon_momentum >> numM', 'muon_momentum>0 & mu_pro>0.5')

eff2 = ROOT.TEfficiency(histo_numM, histo_denM)
eff2.SetTitle('BDT efficiency #mu^{-}[ P(#mu^{-})>0.5 ]; momentum [GeV/c];#epsilon')
eff2.SetName('muon')
eff2.SetMarkerStyle(22)
eff2.SetLineColor(600)
eff2.SetMarkerSize(2)
eff2.SetMarkerColor(600)
eff2.Draw('APL')
# ==================================================================================

histo_denK = ROOT.TH1F('denK', 'denK', len(bins)-1, bins)
histo_numK = ROOT.TH1F('numK', 'numK', len(bins)-1, bins)

kaons_tree.Draw('kaon_momentum >> denK', 'kaon_momentum>0')
kaons_tree.Draw('kaon_momentum >> numK', 'kaon_momentum>0 & ka_pro>0.5')

eff3 = ROOT.TEfficiency(histo_numK, histo_denK)
eff3.SetTitle('Efficiency [ Probability>0.5 ]; momentum [GeV/c];#epsilon')
eff3.SetName('kaon')
eff3.SetMarkerStyle(33)
eff3.SetLineColor(1)
eff3.SetMarkerSize(2)
eff3.SetMarkerColor(1)
eff3.Draw('APL')
# ==================================================================================

histo_denP = ROOT.TH1F('denP', 'denP', len(bins)-1, bins)
histo_numP = ROOT.TH1F('numP', 'numP', len(bins)-1, bins)

protons_tree.Draw('proton_momentum >> denP', 'proton_momentum>0')
protons_tree.Draw('proton_momentum >> numP', 'proton_momentum>0 & ap_pro>0.5')

eff4 = ROOT.TEfficiency(histo_numP, histo_denP)
eff4.SetTitle('BDT efficiency #overline{p}^{-}[ P(#bar{p}^{-})>0.5 ]; momentum [GeV/c];#epsilon')
eff4.SetName('proton')
eff4.SetMarkerStyle(34)
eff4.SetLineColor(416)
eff4.SetMarkerSize(2)
eff4.SetMarkerColor(416)
eff4.Draw('APL')
# ==================================================================================
outfile = ROOT.TFile ("eff.root","RECREATE");
outfile.cd()
eff0.Write()
eff1.Write()
eff2.Write()
eff3.Write()
eff4.Write()
outfile.Close()

In [8]:
from root_numpy import array2root

predclass_el = np.array([])
predictions = model.predict(X_test)

import pandas as pd
f = pd.DataFrame(model.predict_proba(X_test)) # X_test , X_train, X
el = np.array([])
pi = np.array([])
mu = np.array([])
ka = np.array([])
ap = np.array([])

for i in range(len(f.index)):
    if f.iloc[i,:].idxmax() == 0:
        el = np.append(el, f.iloc[i,0])
    elif f.iloc[i,:].idxmax() == 1:
        pi = np.append(pi, f.iloc[i,1])
    elif f.iloc[i,:].idxmax() == 2:
        mu = np.append(mu, f.iloc[i,2])
    elif f.iloc[i,:].idxmax() == 3:
        ka = np.append(ka, f.iloc[i,3])
    else:
        ap = np.append(ap, f.iloc[i,4])      

kwargs = dict(histtype='step', alpha=0.95, normed=True, bins=100, range=(0.0,1),)
fig, ax = plt.subplots()
ax.hist(el, label='e-', **kwargs)
ax.hist(pi, label='pi-', **kwargs)
ax.hist(mu, label='mu-', **kwargs)
ax.hist(ka, label='k-', **kwargs)
ax.hist(ap, label='p-', **kwargs)
leg = ax.legend(loc='best')


variable "ax" not defined.

variable "ax.legend(loc)" not defined.


In [21]:
el.ravel()
pi.ravel()
mu.ravel()
ka.ravel()
ap.ravel()
el_df = pd.DataFrame(electrons,columns=branch_names)
pi_df = pd.DataFrame(pions,columns=branch_names)
mu_df = pd.DataFrame(muons,columns=branch_names)
ka_df = pd.DataFrame(kaons,columns=branch_names)
ap_df = pd.DataFrame(anti_p,columns=branch_names)
momentumArr_el = np.array(el_df.loc[:,'momentum']).ravel()
momentumArr_pi = np.array(pi_df.loc[:,'momentum']).ravel()
momentumArr_mu = np.array(mu_df.loc[:,'momentum']).ravel()
momentumArr_ka = np.array(ka_df.loc[:,'momentum']).ravel()
momentumArr_ap = np.array(ap_df.loc[:,'momentum']).ravel()

import ROOT
def fill_hist(hist, array):
     [hist.Fill(_) for _ in array]
        
EL = ROOT.TH1F('electron', 'electron', 100, 0, 1)
PI = ROOT.TH1F('pion', 'pion', 100, 0, 1) 
MU = ROOT.TH1F('muon', 'muon', 100, 0, 1) 
KA = ROOT.TH1F('kaon', 'kaon', 100, 0, 1) 
AP = ROOT.TH1F('proton', 'proton', 100, 0, 1) 

mom_el = ROOT.TH1F('e_momentum', 'e_momentum', 100, 0, 10) 
mom_pi = ROOT.TH1F('pi_momentum', 'pi_momentum', 100, 0, 10) 
mom_mu = ROOT.TH1F('m_momentum', 'm_momentum', 100, 0, 10) 
mom_ka = ROOT.TH1F('k_momentum', 'k_momentum', 100, 0, 10) 
mom_ap = ROOT.TH1F('p_momentum', 'p_momentum', 100, 0, 10)

fill_hist(EL, np.array(f.iloc[:,0]))
fill_hist(PI, pi)
fill_hist(MU, mu)
fill_hist(KA, ka)
fill_hist(AP, ap)
fill_hist(mom_el, np.array(X_test.loc[:,'momentum']).ravel())
fill_hist(mom_pi, momentumArr_pi)
fill_hist(mom_mu, momentumArr_mu)
fill_hist(mom_ka, momentumArr_ka)
fill_hist(mom_ap, momentumArr_ap)

#for i in momentumArr_el:
#    mom_el.Fill(momentumArr_el)

outfile = ROOT.TFile ("test-prediction.root","RECREATE");
outfile.cd()
EL.Write()
PI.Write()
MU.Write()
KA.Write()
AP.Write()
mom_el.Write()
mom_pi.Write()
mom_mu.Write()
mom_ka.Write()
mom_ap.Write()
outfile.Close()

TROOT::Append:0: RuntimeWarning: Replacing existing TH1: electron (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: pion (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: muon (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: kaon (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: proton (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: e_momentum (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: pi_momentum (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: m_momentum (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: k_momentum (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: p_momentum (Potential memory leak).


In [20]:
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [86]:
import ROOT

def fill_tree(tree, array):
     [tree.Fill() for _ in array]

def fill_hist(hist, array):
     [hist.Fill(_) for _ in array]        

EL = ROOT.TH1F('electron', 'Probability Distributions', 100, 0, 1)
PI = ROOT.TH1F('pion', 'pion', 100, 0, 1) 
MU = ROOT.TH1F('muon', 'muon', 100, 0, 1) 
KA = ROOT.TH1F('kaon', 'kaon', 100, 0, 1) 
AP = ROOT.TH1F('proton', 'proton', 100, 0, 1) 

fill_hist(EL, el)
fill_hist(PI, pi)
fill_hist(MU, mu)
fill_hist(KA, ka)
fill_hist(AP, ap)

EL.SetName('electron_probability')
PI.SetName('pion_probability')
MU.SetName('muon_probability')
KA.SetName('kaon_probability')
AP.SetName('proton_probability')

outfile = ROOT.TFile ("test-prediction.root","RECREATE");
outfile.cd()
EL.Write()
PI.Write()
MU.Write()
KA.Write()
AP.Write()
outfile.Close()

In [46]:
from root_numpy import array2tree, array2hist, fill_hist, rec2array
from root_pandas import to_root
import pandas as pd
from rootpy.plotting import Hist1D, Hist2D
import ROOT
from ROOT import TTree, TFile
from array import array


el_df = pd.DataFrame(electrons,columns=branch_names)
pi_df = pd.DataFrame(pions,columns=branch_names)
mu_df = pd.DataFrame(muons,columns=branch_names)
ka_df = pd.DataFrame(kaons,columns=branch_names)
ap_df = pd.DataFrame(anti_p,columns=branch_names)


el = np.array(f.iloc[:,0]).ravel()
pi = np.array(f.iloc[:,1]).ravel()
mu = np.array(f.iloc[:,2]).ravel()
ka = np.array(f.iloc[:,3]).ravel()
ap = np.array(f.iloc[:,4]).ravel()

#momentumArr_el = np.array(df.loc[df.loc[:,'labels'] == 4, ['momentum']]).ravel()
momentumArr_el = np.array(df.loc[:,'momentum']).ravel()
momentumArr_pi = np.array(df.loc[:,'momentum']).ravel()
momentumArr_mu = np.array(df.loc[:,'momentum']).ravel()
momentumArr_ka = np.array(df.loc[:,'momentum']).ravel()
momentumArr_ap = np.array(df.loc[:,'momentum']).ravel()



el.dtype = [('electron_pro', np.float64)]
pi.dtype = [('pi_pro', np.float64)]
mu.dtype = [('mu_pro', np.float64)]
ka.dtype = [('ka_pro', np.float64)]
ap.dtype = [('ap_pro', np.float64)]
momentumArr_el.dtype = [('electron_momentum', np.float64)]
momentumArr_pi.dtype = [('pion_momentum', np.float64)]
momentumArr_mu.dtype = [('muon_momentum', np.float64)]
momentumArr_ka.dtype = [('kaon_momentum', np.float64)]
momentumArr_ap.dtype = [('proton_momentum', np.float64)]

#array2root(el, 'test-prediction.root', 'output', mode='recreate')
#array2root(pi, 'test-prediction.root', 'output')
#array2root(mu, 'test-prediction.root', 'output')
#array2root(ka, 'test-prediction.root', 'output')
#array2root(ap, 'test-prediction.root', 'output')
array2root(momentumArr_el, 'test-prediction.root', 'output', mode='recreate')
array2root(momentumArr_pi, 'test-prediction.root', 'output')
array2root(momentumArr_mu, 'test-prediction.root', 'output')
array2root(momentumArr_ka, 'test-prediction.root', 'output')
array2root(momentumArr_ap, 'test-prediction.root', 'output')
array2root(el, 'test-prediction.root', 'output', mode='update')
array2root(pi, 'test-prediction.root', 'output')
array2root(mu, 'test-prediction.root', 'output')
array2root(ka, 'test-prediction.root', 'output')
array2root(ap, 'test-prediction.root', 'output')

In [47]:
import ROOT
infile = ROOT.TFile.Open('test-prediction.root', 'read')
infile.cd()
tree = infile.Get('output')

bins = np.array([0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3., 3.5, 4., 4.5, 5.0,]) # variable binning

# =================================================================================
histo_denE = ROOT.TH1F('denE', 'denE', len(bins)-1, bins)
histo_numE = ROOT.TH1F('numE', 'numE', len(bins)-1, bins)

tree.Draw('electron_momentum >> denE', 'electron_momentum>0')
tree.Draw('electron_momentum >> numE', 'electron_momentum>0 & electron_pro>0.5')

eff0 = ROOT.TEfficiency(histo_numE, histo_denE)
eff0.SetTitle('BDT efficiency e^{-}[ P(e^{-})>0.5 ]; momentum [GeV];#epsilon')
eff0.SetName('eff')
eff0.SetMarkerStyle(8)
eff0.SetMarkerColor(1)
eff0.Draw('APL')
# ==================================================================================

histo_denPi = ROOT.TH1F('denPi', 'denPi', len(bins)-1, bins)
histo_numPi = ROOT.TH1F('numPi', 'numPi', len(bins)-1, bins)

tree.Draw('pion_momentum >> denPi', 'pion_momentum>0')
tree.Draw('pion_momentum >> numPi', 'pion_momentum>0 & pi_pro>0.5')

eff1 = ROOT.TEfficiency(histo_numPi, histo_denPi)
eff1.SetTitle('BDT efficiency #pi^{-}[ P(#pi^{-})>0.5 ]; momentum [GeV];#epsilon')
eff1.SetName('eff')
eff1.SetMarkerStyle(8)
eff1.SetMarkerColor(1)
eff1.Draw('APL')
# ==================================================================================

histo_denM = ROOT.TH1F('denM', 'denM', len(bins)-1, bins)
histo_numM = ROOT.TH1F('numM', 'numM', len(bins)-1, bins)

tree.Draw('muon_momentum >> denM', 'muon_momentum>0')
tree.Draw('muon_momentum >> numM', 'muon_momentum>0 & mu_pro>0.5')

eff2 = ROOT.TEfficiency(histo_numM, histo_denM)
eff2.SetTitle('BDT efficiency #mu^{-}[ P(#mu^{-})>0.5 ]; momentum [GeV];#epsilon')
eff2.SetName('eff')
eff2.SetMarkerStyle(8)
eff2.SetMarkerColor(1)
eff2.Draw('APL')
# ==================================================================================

histo_denK = ROOT.TH1F('denK', 'denK', len(bins)-1, bins)
histo_numK = ROOT.TH1F('numK', 'numK', len(bins)-1, bins)

tree.Draw('kaon_momentum >> denK', 'kaon_momentum>0')
tree.Draw('kaon_momentum >> numK', 'kaon_momentum>0 & ka_pro>0.5')

eff3 = ROOT.TEfficiency(histo_numK, histo_denK)
eff3.SetTitle('BDT efficiency k^{-}[ P(k^{-})>0.5 ]; momentum [GeV];#epsilon')
eff3.SetName('eff')
eff3.SetMarkerStyle(8)
eff3.SetMarkerColor(1)
eff3.Draw('APL')
# ==================================================================================

histo_denP = ROOT.TH1F('denP', 'denP', len(bins)-1, bins)
histo_numP = ROOT.TH1F('numP', 'numP', len(bins)-1, bins)

tree.Draw('proton_momentum >> denP', 'proton_momentum>0')
tree.Draw('proton_momentum >> numP', 'proton_momentum>0 & ap_pro>0.5')

eff4 = ROOT.TEfficiency(histo_numP, histo_denP)
eff4.SetTitle('BDT efficiency #overline{p}^{-}[ P(#bar{p}^{-})>0.5 ]; momentum [GeV];#epsilon')
eff4.SetName('eff')
eff4.SetMarkerStyle(8)
eff4.SetMarkerColor(1)
eff4.Draw('APL')
# ==================================================================================
outfile = ROOT.TFile ("eff.root","RECREATE");
outfile.cd()
eff0.Write()
eff1.Write()
eff2.Write()
eff3.Write()
eff4.Write()
outfile.Close()